In [1]:
import os
import pandas as pd

folder_path = "./dataset/labels/val"

file_names = []
file_contents = []

for file_name in os.listdir(folder_path):
    if file_name.endswith(".txt"):
        with open(os.path.join(folder_path, file_name), "r") as file:
            content = file.read()
        file_names.append(file_name)
        file_contents.append(content)

df = pd.DataFrame({"filename": file_names, "file_contents": file_contents})

print(df)


                                     filename  \
0    000db696-cf54-4385-b10b-6b16fbb3f985.txt   
1    0174c4bb-28f5-41e3-a13f-a396badc18bd.txt   
2    01be392f-a46d-4aef-a57e-9cd1a80dd47e.txt   
3    01d4fa0f-97a0-4522-b0d8-134400db2a3e.txt   
4    020380f8-5c5a-4ded-bdf3-9ce3036945b4.txt   
..                                        ...   
602  fde83ad1-3d9e-4a66-8f10-bb6ead510d52.txt   
603  fdf8ddd5-3794-4387-9051-6647f1952b36.txt   
604  fea6191a-f4d4-4cc6-8bf7-5e81bd097209.txt   
605  ff40a703-b1db-48d5-a176-0b50763f9a0a.txt   
606  ff9d37d1-3660-4187-8f57-d31f9c91a3fd.txt   

                                         file_contents  
0    0 0.3916015625 0.5439453125 0.166015625 0.4667...  
1    0 0.28466796875 0.42236328125 0.2666015625 0.4...  
2    0 0.60888671875 0.728515625 0.1728515625 0.234...  
3    0 0.3115234375 0.46484375 0.189453125 0.425781...  
4    0 0.35302734375 0.57568359375 0.1533203125 0.2...  
..                                                 ...  
602  0 0.355

In [2]:
import numpy as np
def iou(box1, box2):
    x11, y11, w1, h1 = box1
    x21, y21, w2, h2 = box2
    assert w1 * h1 > 0
    assert w2 * h2 > 0
    x12, y12 = x11 + w1, y11 + h1
    x22, y22 = x21 + w2, y21 + h2

    area1, area2 = w1 * h1, w2 * h2
    xi1, yi1, xi2, yi2 = max([x11, x21]), max([y11, y21]), min([x12, x22]), min([y12, y22])
    
    if xi2 <= xi1 or yi2 <= yi1:
        return 0
    else:
        intersect = (xi2-xi1) * (yi2-yi1)
        union = area1 + area2 - intersect
        return intersect / union

In [3]:
def translate_bbox(bbox):
    img_size = 1024 # rsna defualt image size
    
    top_left_x = bbox[0]
    top_left_y = bbox[1]
    absolute_w = bbox[2]
    absolute_h = bbox[3]

    relative_w = absolute_w / img_size
    relative_h = absolute_h / img_size
    
    relative_x = top_left_x / img_size + relative_w / 2
    relative_y = top_left_y / img_size + relative_h / 2
    
    return relative_x, relative_y, relative_w, relative_h
    
def revert_bbox(rx, ry, rw, rh):
    img_size = 1024 # rsna defualt image size
    
    x = (rx-rw/2)*img_size
    y = (ry-rh/2)*img_size
    w = rw*img_size
    h = rh*img_size
    
    return x, y, w, h

In [4]:
def evaluate(filename, xywh):
    target_str = df[df['filename'] == filename]
    if target_str == '':
        return 0
    elements = target_str.split()
    sublists = [elements[i:i+5] for i in range(0, len(elements), 5)]
    bbox_targets = [tuple(map(float, sublist[-4:])) for sublist in sublists]
    result = []
    for bbox_target in bbox_targets:
        bbox1 = translate_bbox(bbox_target)
        bbox2 = translate_bbox(xywh)
        result.append(iou(bbox1, bbox2))

    return max(result)
    

In [5]:
thresholds = [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75]
def average_precision(filename, xywh):
    result = evaluate(filename, xywh)
    precision_values = []
    for t in thresholds:
        tp = 0
        fn = 0
        fp = 0
        for res in result:
            if res >= t:
                tp += 1
            else:
                fn += 1
        fp += len(result) - tp
        precision_values = tp / (tp + fp + fn)
            
    return sum(precision_values) / len(thresholds)